<h3>Načitanie knižníc</h3>

In [51]:
import numpy as np
import random

<h3>Definovanie premenných</h3>

In [52]:
# Parametre portfólia
NUM_STOCKS = 10  # Počet akcií
POPULATION_SIZE = 100  # Veľkosť populácie
GENERATIONS = 50  # Počet generácií
MUTATION_RATE = 0.1  # Miera mutácie

# Generovanie náhodných ročných výnosov a rizík (volatilita)
np.random.seed(42)
expected_returns = np.array([0.9457, 0.2943, 0.2498, 0.2478, 0.2254, 0.7197, 0.5089, 0.4734, 0.1813, 0.1622])  # NVDA, AAPL, MSFT, META, GOOGL, TSLA, LLY, AVGO, AMZN, BRK.B
risks = np.array([0.3991, 0.2123, 0.2115, 0.3299, 0.1686, 0.4852, 0.3338, 0.2761, 0.1973, 0.1722 ])  # Očakávané riziká (volatilita)

<h3>Definovanie fitness funkcie</h3>

In [53]:
def fitness_function(weights, expected_returns, risks):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(np.diag(risks**2), weights)))
    return portfolio_return - portfolio_risk

<h3>Normalizovanie váh</h3>

In [54]:
def normalize_weights(weights):
    return weights / np.sum(weights)

<h3>Inicializácia populácie</h3>

In [55]:
def generate_population(size):
    return [normalize_weights(np.random.rand(NUM_STOCKS)) for _ in range(size)]

<h3>Metódy selekcie</h3>

In [56]:
# Turnajová selekcia
def tournament_selection(population, expected_returns, risks):
    tournament = random.sample(population, 5)
    return max(tournament, key=lambda ind: fitness_function(ind, expected_returns, risks))

# Ruletový výber
def roulette_selection(population, expected_returns, risks):
    fitness_values = np.array([fitness_function(ind, expected_returns, risks) for ind in population])
    fitness_sum = np.sum(fitness_values)
    selection_probs = fitness_values / fitness_sum
    return population[np.random.choice(len(population), p=selection_probs)]

# Výber podľa poradia (Rank Selection)
def rank_selection(population, expected_returns, risks):
    fitness_values = np.array([fitness_function(ind, expected_returns, risks) for ind in population])
    ranked_indices = np.argsort(-fitness_values)
    selection_probs = 1 / (np.arange(len(population)) + 1)
    selection_probs /= np.sum(selection_probs)
    return population[np.random.choice(len(population), p=selection_probs)]

<h3> Funkcie mutácie a kríženia</h3>

In [57]:
# Kríženie
def crossover(parent1, parent2):
    point = random.randint(1, NUM_STOCKS - 1)
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return normalize_weights(child1), normalize_weights(child2)

# Mutácia
def mutate(individual):
    if random.random() < MUTATION_RATE:
        idx = random.randint(0, NUM_STOCKS - 1)
        individual[idx] = random.uniform(0.0, 1.0)
        return normalize_weights(individual)
    return individual

<h3>Implementácia genetického algoritmu</h3>

In [58]:
def genetic_algorithm(selection_method, expected_returns, risks):
    population = generate_population(POPULATION_SIZE)
    
    for generation in range(GENERATIONS):
        new_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1 = selection_method(population, expected_returns, risks)
            parent2 = selection_method(population, expected_returns, risks)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1))
            new_population.append(mutate(child2))
        
        population = new_population
        
        best_solution = max(population, key=lambda ind: fitness_function(ind, expected_returns, risks))
        yield generation, best_solution, fitness_function(best_solution, expected_returns, risks)

<h3>Porovnávanie metód selekcie</h3>

In [59]:
# Porovnanie selekčných metód
def compare_selections():
    methods = {
        'Turnajová selekcia': tournament_selection,
        'Ruletový výber': roulette_selection,
        'Výber podľa poradia': rank_selection
    }

    results = {}
    
    for method_name, method in methods.items():
        print(f"\n{method_name}:\n" + "=" * 30)
        best_portfolio = None
        best_fitness = float('-inf')
        best_return = 0
        best_risk = 0
        
        for generation, solution, fitness in genetic_algorithm(method, expected_returns, risks):
            # Vypočítame výnos a riziko samostatne pre najlepšie portfólio
            portfolio_return = np.dot(solution, expected_returns)
            portfolio_risk = np.sqrt(np.dot(solution.T, np.dot(np.diag(risks**2), solution)))
            print(f"Generácia {generation + 1}: Najlepšie váhy: {solution}, Výnos: {portfolio_return:.4f}, Riziko: {portfolio_risk:.4f}")
            
            # Uchovávame najlepšie portfólio podľa fitness hodnoty (rozdiel výnosu a rizika)
            if fitness > best_fitness:
                best_fitness = fitness
                best_portfolio = solution
                best_return = portfolio_return
                best_risk = portfolio_risk
        
        results[method_name] = (best_portfolio, best_return, best_risk)

    print("\nNajlepšie výsledky:")
    for method, (portfolio, portfolio_return, portfolio_risk) in results.items():
        print(f"{method}: Výnos: {portfolio_return:.4f}, Riziko: {portfolio_risk:.4f}, Váhy: {portfolio}")


<h3>Spustenie kódu</h3>

In [60]:
compare_selections()


Turnajová selekcia:
Generácia 1: Najlepšie váhy: [0.22235371 0.0216361  0.14053078 0.05611753 0.04940564 0.218876
 0.00504449 0.20822846 0.02501585 0.05279144], Výnos: 0.5486, Riziko: 0.1545
Generácia 2: Najlepšie váhy: [0.35642603 0.08771118 0.02312959 0.1206011  0.04599865 0.07987029
 0.1013358  0.10038157 0.04889531 0.03565048], Výnos: 0.5801, Riziko: 0.1606
Generácia 3: Najlepšie váhy: [0.52649776 0.01296808 0.08423031 0.03363531 0.03701473 0.13118829
 0.00302353 0.12480644 0.01499382 0.03164175], Výnos: 0.7023, Riziko: 0.2234
Generácia 4: Najlepšie váhy: [0.46362946 0.01141958 0.07417249 0.02961897 0.03259485 0.1783864
 0.00411131 0.16970853 0.0203882  0.0159702 ], Výnos: 0.6921, Riziko: 0.2105
Generácia 5: Najlepšie váhy: [0.47678996 0.01174374 0.07226719 0.02885813 0.03175758 0.17380411
 0.0040057  0.16534915 0.01986448 0.01555997], Výnos: 0.6982, Riziko: 0.2140
Generácia 6: Najlepšie váhy: [0.50072542 0.01233329 0.08010719 0.03198884 0.03520284 0.19265947
 0.00444027 0.1042744